In [59]:
databook_path

'Marktmodell/scenarios/master.xlsm'

In [1]:
## TODO index columns where needed

## Package Imports

In [2]:
import numpy as np
import pulp
import pandas as pd
import os
import openpyxl
import shutil

## DONE Parameters

In [3]:
directory = "Marktmodell"
scenario_folder = "scenarios"
output_folder = "outputs"

## DONE Data Imports

### DONE Scenario Definition

In [4]:
scenario = "master"
databook_path = os.path.join(directory, scenario_folder, scenario+".xlsm")
output_path = os.path.join(directory, output_folder, scenario+".xlsx")
output_template_path = os.path.join(directory, output_folder, "template.xlsx")

### DONE Cycles

In [5]:
cycles = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG > Price Curve",
    skiprows = 4,
    index_col = 0
).columns.tolist()[:-1]
cycles

['Aug-21',
 'Sep-21',
 'Okt-21',
 'Nov-21',
 'Dez-21',
 'Jan-22',
 'Feb-22',
 'Mrz-22',
 'Apr-22',
 'Mai-22',
 'Jun-22',
 'Jul-22',
 'Aug-22',
 'Sep-22',
 'Okt-22',
 'Nov-22',
 'Dez-22']

### DONE Regions

In [6]:
regions_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Regions > Index",
    skiprows = 4,
    index_col = 1
)
regions_df

,Region,Master,Comments,RegionID
Index,,,,
1,Deutschland,NaN,NaN,Region_001
2,Frankreich,NaN,NaN,Region_002
3,Niederlande,NaN,NaN,Region_003
4,Schweiz,NaN,NaN,Region_004


In [7]:
regions_demand_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Regions > Demand",
    skiprows = 4,
    index_col = 1
)
regions_demand_df

,Region,Aug-21,Sep-21,Okt-21,Nov-21,Dez-21,Jan-22,Feb-22,Mrz-22,Apr-22,Mai-22,Jun-22,Jul-22,Aug-22,Sep-22,Okt-22,Nov-22,Dez-22,Source/Comments,DemandID
Index,,,,,,,,,,,,,,,,,,,,
1,Deutschland,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Demand_001
2,Frankreich,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Demand_002
3,Niederlande,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Demand_003
4,Schweiz,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Demand_004


In [8]:
regions_production_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Regions > Production",
    skiprows = 4,
    index_col = 1
)
regions_production_df

,Region,Aug-21,Sep-21,Okt-21,Nov-21,Dez-21,Jan-22,Feb-22,Mrz-22,Apr-22,Mai-22,Jun-22,Jul-22,Aug-22,Sep-22,Okt-22,Nov-22,Dez-22,Source/Comments,ProductionID
Index,,,,,,,,,,,,,,,,,,,,
1,Deutschland,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Production_001
2,Frankreich,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Production_002
3,Niederlande,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Production_003
4,Schweiz,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Production_004


### DONE Piped Importers

In [9]:
piped_importers_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Piped Importers > Index",
    skiprows = 4,
    index_col = 1
)
piped_importers_df

,Importer,Comments,PipedImporterID
Index,,,
1,Norwegen,NaN,PipedImporter_001
2,Algerien,NaN,PipedImporter_002


In [10]:
piped_importers_production_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Piped Importers > Production",
    skiprows = 4,
    index_col = 1
)
piped_importers_production_df

,Importer,Aug-21,Sep-21,Okt-21,Nov-21,Dez-21,Jan-22,Feb-22,Mrz-22,Apr-22,Mai-22,Jun-22,Jul-22,Aug-22,Sep-22,Okt-22,Nov-22,Dez-22,Source/Comments,PipedImporterProductionID
Index,,,,,,,,,,,,,,,,,,,,
1,Norwegen,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PipedImporterProduction_001
2,Algerien,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PipedImporterProduction_002


In [11]:
piped_importers_connections_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Piped Importers > Connections",
    skiprows = 4,
    index_col = 8
)
piped_importers_connections_df

,Importer,Importer Index,Region,Region Index,Name,Connection Code,Connection Index,Comments
PipedImporterConnectionID,,,,,,,,
PipedImporterConnection_001_001_001,Norwegen,1,Deutschland,1,Import Nor-DE I,1001,1,NaN
PipedImporterConnection_001_002_001,Norwegen,1,Frankreich,2,Import Nor-FR,1002,1,NaN
PipedImporterConnection_001_003_001,Norwegen,1,Niederlande,3,Import Nor-NL,1003,1,NaN
PipedImporterConnection_002_002_001,Algerien,2,Frankreich,2,Import Alg-FR,2002,1,NaN
PipedImporterConnection_002_004_001,Algerien,2,Schweiz,4,Import Alg-CH,2004,1,NaN
PipedImporterConnection_001_001_002,Norwegen,1,Deutschland,1,Import Nor-DE II,1001,2,NaN


In [12]:
piped_importers_connections_data_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Piped Importers > Data",
    skiprows = 4,
    index_col = [0,1],
    usecols = "B:C,E:U"
).fillna(0)
temp_index = piped_importers_connections_data_df.index.names
piped_importers_connections_data_df = piped_importers_connections_data_df.reset_index().dropna().set_index(temp_index)
piped_importers_connections_data_df = piped_importers_connections_data_df.reset_index().merge(
    piped_importers_connections_df[["Name", "Importer"]],
    left_on = "Piped Import Connection",
    right_on = "Name",
    how = "left"
)
piped_importers_connections_data_df.drop(columns = ["Name"], inplace = True)
piped_importers_connections_data_df = piped_importers_connections_data_df.reset_index().dropna().set_index(temp_index)
piped_importers_connections_data_df


index  Aug-21  Sep-21  \
Piped Import Connection Metric                                             
Import Nor-DE I         Capacity - Min                 0     0.0     0.0   
                        Capacity - Max                 1    10.0     0.0   
                        Cost to Market - fixed         2     0.0     0.0   
                        Cost to Market - variable      3     0.5     0.0   
Import Nor-FR           Capacity - Min                 4     0.0     0.0   
                        Capacity - Max                 5     7.0     0.0   
                        Cost to Market - fixed         6     0.0     0.0   
                        Cost to Market - variable      7     0.3     0.0   
Import Nor-NL           Capacity - Min                 8     0.5     0.0   
                        Capacity - Max                 9     1.0     0.0   
                        Cost to Market - fixed        10     0.0     0.0   
                        Cost to Market - variable     11     0.3     0.0   
Import Alg-FR           Capacity - Min                12     0.0     0.0   
                        Capacity - Max                13     2.0     0.0   
                        Cost to Market - fixed        14     0.0     0.0   
                        Cost to Market - variable     15     0.2     0.0   
Import Alg-CH           Capacity - Min                16     0.0     0.0   
                        Capacity - Max                17     4.0     0.0   
                        Cost to Market - fixed        18     0.0     0.0   
                        Cost to Market - variable     19     0.3     0.0   
Import Nor-DE II        Capacity - Min                20     0.0     0.0   
                        Capacity - Max                21     5.0     0.0   
                        Cost to Market - fixed        22     0.0     0.0   
                        Cost to Market - variable     23     0.7     0.0   

                                                   Okt-21  Nov-21  Dez-21  \
Piped Import Connection Metric                                              
Import Nor-DE I         Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   
Import Nor-FR           Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   
Import Nor-NL           Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   
Import Alg-FR           Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   
Import Alg-CH           Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   
Import Nor-DE II        Capacity - Min                0.0     0.0     0.0   
                        Capacity - Max                0.0     0.0     0.0   
                        Cost to Market - fixed        0.0     0.0     0.0   
                        Cost to Market - variable     0.0     0.0     0.0   

                                                   Jan-22  Feb-22  Mrz-

### DONE LNG Terminals

In [13]:
lng_importers_df = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG Importers > Index",
    skiprows = 4,
    index_col = 1
)
lng_importers_df

,Terminal,Comments,LNGImporterID
Index,,,
1,GATE,NaN,LNGImporter_001
2,Dunkirk,NaN,LNGImporter_002


In [14]:
lng_importers_connections_df = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG Importers > Connections",
    skiprows = 4,
    index_col = 8
)
lng_importers_connections_df

,Terminal,Terminal Index,Region,Region Index,Name,Connection Code,Connection Index,Comments
LNGImporterConnectionID,,,,,,,,
LNGImporterConnection_001_003_001,GATE,1,Niederlande,3,GATE NL,1003,1,NaN
LNGImporterConnection_001_001_001,GATE,1,Deutschland,1,GATE DE,1001,1,NaN
LNGImporterConnection_002_002_001,Dunkirk,2,Frankreich,2,Dunkirk FR,2002,1,NaN


In [15]:
lng_importers_connections_data_df = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG Importers > Data",
    skiprows = 4,
    index_col = [0,1],
    usecols = "B:C,E:U"
).fillna(0)
temp_index = lng_importers_connections_data_df.index.names
lng_importers_connections_data_df = lng_importers_connections_data_df.reset_index().dropna().set_index(temp_index)
lng_importers_connections_data_df

Aug-21  Sep-21  Okt-21  \
LNG Import Connection Metric                                              
GATE NL               Capacity - Min               0.00     0.0     0.0   
                      Capacity - Max              10.00     0.0     0.0   
                      Cost to Market - fixed       0.00     0.0     0.0   
                      Cost to Market - variable    0.40     0.0     0.0   
GATE DE               Capacity - Min               0.00     0.0     0.0   
                      Capacity - Max              10.00     0.0     0.0   
                      Cost to Market - fixed       0.00     0.0     0.0   
                      Cost to Market - variable    0.20     0.0     0.0   
Dunkirk FR            Capacity - Min               0.00     0.0     0.0   
                      Capacity - Max              10.00     0.0     0.0   
                      Cost to Market - fixed       0.00     0.0     0.0   
                      Cost to Market - variable    0.35     0.0     0.0   

                                                 Nov-21  Dez-21  Jan-22  \
LNG Import Connection Metric                                              
GATE NL               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
GATE DE               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
Dunkirk FR            Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   

                                                 Feb-22  Mrz-22  Apr-22  \
LNG Import Connection Metric                                              
GATE NL               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
GATE DE               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
Dunkirk FR            Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   

                                                 Mai-22  Jun-22  Jul-22  \
LNG Import Connection Metric                                              
GATE NL               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
GATE DE               Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0   
                      Cost to Market - fixed        0.0     0.0     0.0   
                      Cost to Market - variable     0.0     0.0     0.0   
Dunkirk FR            Capacity - Min                0.0     0.0     0.0   
                      Capacity - Max                0.0     0.0     0.0

### DONE Region Connections

In [16]:
connections_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Connections > Index",
    skiprows = 4,
    index_col = 8
)
connections_df

,Origin,Origin Index,Destination,Destination Index,Name,Connection Code,Connection Index,Comments
TransitID,,,,,,,,
Transit_001_002_001,Deutschland,1,Frankreich,2,DE-FR I,1002,1,NaN
Transit_002_001_001,Frankreich,2,Deutschland,1,FR-DE,2001,1,NaN
Transit_001_003_001,Deutschland,1,Niederlande,3,DE-NL,1003,1,NaN
Transit_003_001_001,Niederlande,3,Deutschland,1,NL-DE,3001,1,NaN
Transit_001_004_001,Deutschland,1,Schweiz,4,DE-CH,1004,1,NaN
Transit_002_003_001,Frankreich,2,Niederlande,3,FR-NL,2003,1,NaN
Transit_003_002_001,Niederlande,3,Frankreich,2,NL-FR,3002,1,NaN
Transit_002_004_001,Frankreich,2,Schweiz,4,FR-CH,2004,1,NaN
Transit_004_002_001,Schweiz,4,Frankreich,2,CH-FR,4002,1,NaN


In [17]:
connections_data_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Connections > Data",
    skiprows = 4,
    index_col = [0,1],
    usecols = "B:C,E:U"
).fillna(0)
temp_index = connections_data_df.index.names
connections_data_df = connections_data_df.reset_index().dropna().set_index(temp_index)
connections_data_df

Aug-21  Sep-21  Okt-21  Nov-21  Dez-21  \
Transit Connection Metric                                                      
DE-FR I            Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max       0.50     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.12     0.0     0.0     0.0     0.0   
FR-DE              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      10.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.10     0.0     0.0     0.0     0.0   
DE-NL              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      10.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.20     0.0     0.0     0.0     0.0   
NL-DE              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      10.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.30     0.0     0.0     0.0     0.0   
DE-CH              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      20.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.50     0.0     0.0     0.0     0.0   
FR-NL              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max       2.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed      20.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.10     0.0     0.0     0.0     0.0   
NL-FR              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      20.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.20     0.0     0.0     0.0     0.0   
FR-CH              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max       2.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.20     0.0     0.0     0.0     0.0   
CH-FR              Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      30.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.20     0.0     0.0     0.0     0.0   
DE-FR II           Capacity - Min       0.00     0.0     0.0     0.0     0.0   
                   Capacity - Max      40.00     0.0     0.0     0.0     0.0   
                   Tariff - fixed       0.00     0.0     0.0     0.0     0.0   
                   Tariff - variable    0.15     0.0     0.0     0.0     0.0   

                                      Jan-22  Feb-22  Mrz-22  Apr-22  Mai-22  \
Transit Connection Metric                                                      
DE-FR I            Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
FR-DE              Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed    

### DONE Storage

In [18]:
storage_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Storage > Index",
    skiprows = 4,
    index_col = 1
)
storage_df

,Storage,WGV (bcm),Comments,StorageID
Storage Index,,,,
1,Storage A,NaN,NaN,Storage_001
2,Storage B,NaN,NaN,Storage_002


In [19]:
storage_connections_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Storage > Connections",
    skiprows = 4,
    index_col = 8
)
storage_connections_df

,Storage,Storage Index,Region,Region Index,Name,Connection Code,Connection Index,Comments
StorageConnectionID,,,,,,,,
StorageConnection_001_004_001,Storage A,1,Schweiz,4,St A CH,1004,1,NaN
StorageConnection_001_001_001,Storage A,1,Deutschland,1,St A DE I,1001,1,NaN
StorageConnection_002_002_001,Storage B,2,Frankreich,2,St B FR,2002,1,NaN
StorageConnection_001_001_002,Storage A,1,Deutschland,1,St A DE II,1001,2,NaN


In [20]:
storage_connections_data_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Storage > Data",
    skiprows = 4,
    index_col = [0,1],
    usecols = "B:C,E:U"
).fillna(0)
temp_index = storage_connections_data_df.index.names
storage_connections_data_df = storage_connections_data_df.reset_index().dropna().set_index(temp_index)
storage_connections_data_df

Aug-21  Sep-21  Okt-21  Nov-21  Dez-21  \
Storage Connection Metric                                                      
St A CH            Capacity - Min       -3.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        3.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE I          Capacity - Min       -4.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        2.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St B FR            Capacity - Min       -1.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        6.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE II         Capacity - Min       -2.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        2.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   

                                      Jan-22  Feb-22  Mrz-22  Apr-22  Mai-22  \
Storage Connection Metric                                                      
St A CH            Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE I          Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St B FR            Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE II         Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   

                                      Jun-22  Jul-22  Aug-22  Sep-22  Okt-22  \
Storage Connection Metric                                                      
St A CH            Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE I          Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St B FR            Capacity - Min        0.0     0.0     0.0     0.0     0.0   
                   Capacity - Max        0.0     0.0     0.0     0.0     0.0   
                   Tariff - fixed        0.0     0.0     0.0     0.0     0.0   
                   Tariff - variable     0.0     0.0     0.0     0.0     0.0   
St A DE II         Capacity - Min   

### DONE LNG Supply

In [21]:
lng_supply_curve_df = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG > Price Curve",
    skiprows = 4,
    index_col = 0
)
lng_supply_curve_df

,Aug-21,Sep-21,Okt-21,Nov-21,Dez-21,Jan-22,Feb-22,Mrz-22,Apr-22,Mai-22,Jun-22,Jul-22,Aug-22,Sep-22,Okt-22,Nov-22,Dez-22,Source/Comments
Supply (Mt LNG/month),,,,,,,,,,,,,,,,,,
50,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,4.647862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,4.730151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,5.413859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,5.932805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,6.794467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,7.562619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,7.695087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,8.462843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
lng_other_demand_dict = pd.read_excel(
    io = databook_path,
    sheet_name = "LNG > Other Demand",
    skiprows = 4,
    index_col = 0
).fillna(0).iloc[0].to_dict()
lng_other_demand_dict

{'Aug-21': 85.0,
 'Sep-21': 0.0,
 'Okt-21': 0.0,
 'Nov-21': 0.0,
 'Dez-21': 0.0,
 'Jan-22': 0.0,
 'Feb-22': 0.0,
 'Mrz-22': 0.0,
 'Apr-22': 0.0,
 'Mai-22': 0.0,
 'Jun-22': 0.0,
 'Jul-22': 0.0,
 'Aug-22': 0.0,
 'Sep-22': 0.0,
 'Okt-22': 0.0,
 'Nov-22': 0.0,
 'Dez-22': 0.0,
 'Source/Comments': 0.0}

### DONE Cycle Lengths

In [23]:
cycles_days_dict = pd.read_excel(
    io = databook_path,
    sheet_name = "Cycles",
    skiprows = 4,
    index_col = 0
).fillna(0).iloc[0].to_dict()
cycles_days_dict

{'Aug-21': 31,
 'Sep-21': 30,
 'Okt-21': 31,
 'Nov-21': 30,
 'Dez-21': 31,
 'Jan-22': 31,
 'Feb-22': 28,
 'Mrz-22': 31,
 'Apr-22': 30,
 'Mai-22': 31,
 'Jun-22': 30,
 'Jul-22': 31,
 'Aug-22': 31,
 'Sep-22': 30,
 'Okt-22': 31,
 'Nov-22': 30,
 'Dez-22': 31}

### DONE Unit Conversions

In [24]:
unit_conversions_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Unit Conversions",
    skiprows = 6,
    index_col = 0
)
unit_conversions_df

,bcm,mcm,cm,kWh,MWh,GWh,TWh,MMBTU,Mt LNG
bcm,1.000000e+00,1.000000e+03,1.000000e+09,9.769444e+09,9.769444e+06,9769.444444,9.769444e+00,3.333473e+07,7.518797e-01
mcm,1.000000e-03,1.000000e+00,1.000000e+06,9.769444e+06,9.769444e+03,9.769444,9.769444e-03,3.333473e+04,7.518797e-04
cm,1.000000e-09,1.000000e-06,1.000000e+00,9.769444e+00,9.769444e-03,0.000010,9.769444e-09,3.333473e-02,7.518797e-10
kWh,1.023600e-07,1.023600e-04,1.023600e-01,1.000000e+00,1.000000e-03,0.000001,1.000000e-09,3.412142e-03,7.696238e-11
MWh,1.023600e-10,1.023600e-07,1.023600e-04,1.000000e-03,1.000000e+00,0.001000,1.000000e-06,3.412142e+00,7.696238e-08
GWh,1.023600e-13,1.023600e-10,1.023600e-07,1.000000e-06,1.000000e-03,1.000000,1.000000e-03,3.412142e+03,7.696238e-05
TWh,1.023600e-16,1.023600e-13,1.023600e-10,1.000000e-09,1.000000e-06,0.001000,1.000000e+00,3.412142e+06,7.696238e-02
MMBTU,2.999874e-08,2.999874e-05,2.999874e+01,2.930711e+02,2.930711e-01,0.000293,2.930711e-07,1.000000e+00,2.255545e-08
Mt LNG,1.330000e+00,1.330000e+03,1.330000e+09,1.299336e+10,1.299336e+07,12993.361111,1.299336e+01,4.433519e+07,1.000000e+00


### DONE Forex Conversions

In [25]:
forex_conversions_df = pd.read_excel(
    io = databook_path,
    sheet_name = "Forex Conversions",
    skiprows = 4,
    index_col = 0
)
forex_conversions_df

,Aug-21,Sep-21,Okt-21,Nov-21,Dez-21,Jan-22,Feb-22,Mrz-22,Apr-22,Mai-22,Jun-22,Jul-22,Aug-22,Sep-22,Okt-22,Nov-22,Dez-22,Source/Comments
Currency,,,,,,,,,,,,,,,,,,
USD,1.177023,1.177358,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,1.158342,OFX
GBP,0.853156,0.856443,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,0.849206,OFX


## DONE Data Initialisation

### DONE Temporary hardcode

In [26]:
cycle_index = 0

### DONE Cycle Definitions

In [27]:
# Cycle label
cycle = cycles[cycle_index]
# Days in cycle
cycle_days = cycles_days_dict[cycle]

### DONE Define Output Sets

In [28]:
output_demand_df = regions_demand_df.drop(columns = ["Source/Comments", "DemandID"]).set_index("Region")
output_production_df = regions_production_df.drop(columns = ["Source/Comments", "ProductionID"]).set_index("Region")
output_piped_importers_df = pd.DataFrame()
output_lng_importers_df = pd.DataFrame()
output_connections_df = pd.DataFrame()
output_prices_df = pd.DataFrame()
output_supply_mix_df = pd.DataFrame(
    0,
    pd.MultiIndex.from_product(
    [
        output_demand_df.index.values.tolist(),
        ["Production", "Piped Imports", "LNG Imports", "Imports"]
    ],
    names = ["Region", "Type"]
    ),
    []
)

### DONE Model Variables

#### DONE Piped Importer Connections

In [29]:
piped_importers_connections_dict = {}
for piped_importers_connection_index, piped_importers_connection in piped_importers_connections_df.iterrows():
    piped_importers_connections_dict[piped_importers_connection_index] = pulp.LpVariable(
        piped_importers_connection["Name"],
        lowBound = 0,
        upBound = (
            piped_importers_connections_data_df.loc[
                piped_importers_connection["Name"],
                "Capacity - Max"]
            [cycle]
            - piped_importers_connections_data_df.loc[
                piped_importers_connection["Name"],
                "Capacity - Min"]
            [cycle]
        ) * cycle_days
    )
piped_importers_connections_dict

{'PipedImporterConnection_001_001_001': Import_Nor_DE_I,
 'PipedImporterConnection_001_002_001': Import_Nor_FR,
 'PipedImporterConnection_001_003_001': Import_Nor_NL,
 'PipedImporterConnection_002_002_001': Import_Alg_FR,
 'PipedImporterConnection_002_004_001': Import_Alg_CH,
 'PipedImporterConnection_001_001_002': Import_Nor_DE_II}

#### DONE LNG Importer Connections

In [30]:
lng_importers_connections_dict = {}
for lng_importers_connection_index, lng_importers_connection in lng_importers_connections_df.iterrows():
    lng_importers_connections_dict[lng_importers_connection_index] = pulp.LpVariable(
        lng_importers_connection["Name"],
        lowBound = 0,
        upBound = (
            lng_importers_connections_data_df.loc[
                lng_importers_connection["Name"],
                "Capacity - Max"]
            [cycle]
            - lng_importers_connections_data_df.loc[
                lng_importers_connection["Name"],
                "Capacity - Min"]
            [cycle]
        ) * cycle_days
    )
lng_importers_connections_dict

{'LNGImporterConnection_001_003_001': GATE_NL,
 'LNGImporterConnection_001_001_001': GATE_DE,
 'LNGImporterConnection_002_002_001': Dunkirk_FR}

#### DONE Region Connections

In [31]:
connections_dict = {}
for connection_index, connection in connections_df.iterrows():
    connections_dict[connection_index] = pulp.LpVariable(
        connection["Name"],
        lowBound = 0,
        upBound = (
            connections_data_df.loc[
                connection["Name"],
                "Capacity - Max"]
            [cycle]
            - connections_data_df.loc[
                connection["Name"],
                "Capacity - Min"]
            [cycle]
        ) * cycle_days
    )
connections_dict

{'Transit_001_002_001': DE_FR_I,
 'Transit_002_001_001': FR_DE,
 'Transit_001_003_001': DE_NL,
 'Transit_003_001_001': NL_DE,
 'Transit_001_004_001': DE_CH,
 'Transit_002_003_001': FR_NL,
 'Transit_003_002_001': NL_FR,
 'Transit_002_004_001': FR_CH,
 'Transit_004_002_001': CH_FR,
 'Transit_001_002_002': DE_FR_II}

#### DONE Storage Connections

In [32]:
storage_connections_dict = {}
for storage_connection_index, storage_connection in storage_connections_df.iterrows():
    storage_connections_dict[storage_connection_index] = pulp.LpVariable(
        storage_connection["Name"],
        lowBound = 0,
        upBound = (
            storage_connections_data_df.loc[
                storage_connection["Name"],
                "Capacity - Max"]
            [cycle]
            - storage_connections_data_df.loc[
                storage_connection["Name"],
                "Capacity - Min"]
            [cycle]
        ) * cycle_days
    )
storage_connections_dict

{'StorageConnection_001_004_001': St_A_CH,
 'StorageConnection_001_001_001': St_A_DE_I,
 'StorageConnection_002_002_001': St_B_FR,
 'StorageConnection_001_001_002': St_A_DE_II}

### DONE Regions

In [33]:
'''
Dictionary for country: [factor, connection, fixed tariff, variable tariff, lng source switch, name, source] pairings
'''
regions_supply_dict = {}
# Dictionary for country: demand pairings
regions_demand_dict = {}
for region_index, region in regions_df.iterrows():
    # Demand
    region_demand = regions_demand_df.loc[region_index][cycle]
    regions_demand_dict[region["Region"]] = region_demand * cycle_days
    
    # Production
    region_production = regions_production_df.loc[region_index][cycle]
    regions_supply_dict[region["Region"]] = [[1, region_production * cycle_days, 0, 0, 0, f"Production {region['Region']}", ""]]
    
    # Piped imports
    piped_importer_list = piped_importers_connections_df[piped_importers_connections_df["Region Index"] == region_index]["Name"].to_dict()
    for piped_importer_id, piped_importer in piped_importer_list.items():
        # Min flow
        regions_supply_dict[region["Region"]].append([1, piped_importers_connections_data_df.loc[piped_importer, "Capacity - Min"][cycle] * cycle_days, 0, 0, 0, f"{piped_importer} - Min", ""])
        # Variable flow
        regions_supply_dict[region["Region"]].append(
            [
                1,
                piped_importers_connections_dict[piped_importer_id],
                piped_importers_connections_data_df.loc[piped_importer, "Cost to Market - fixed"][cycle],
                piped_importers_connections_data_df.loc[piped_importer, "Cost to Market - variable"][cycle] * unit_conversions_df.loc["mcm"]["MWh"],
                0,
                piped_importer,
                "Pipe"
            ]
        )
    
    # LNG imports
    lng_importer_list = lng_importers_connections_df[lng_importers_connections_df["Region Index"] == region_index]["Name"].to_dict()
    for lng_importer_id, lng_importer in lng_importer_list.items():
        # Min flow
        regions_supply_dict[region["Region"]].append([1, lng_importers_connections_data_df.loc[lng_importer, "Capacity - Min"][cycle] * cycle_days, 0, 0, 0, f"{lng_importer} - Min", ""])
        # Variable flow
        regions_supply_dict[region["Region"]].append(
            [
                1,
                lng_importers_connections_dict[lng_importer_id],
                lng_importers_connections_data_df.loc[lng_importer, "Cost to Market - fixed"][cycle],
                lng_importers_connections_data_df.loc[lng_importer, "Cost to Market - variable"][cycle] * unit_conversions_df.loc["mcm"]["MWh"],
                1,
                lng_importer,
                "LNG"
            ]
        )
        
    # Region connections (imports)
    import_list = connections_df[connections_df["Destination Index"] == region_index]["Name"].to_dict()
    for importer_id, importer in import_list.items():
        # Min flow
        regions_supply_dict[region["Region"]].append([1, connections_data_df.loc[importer, "Capacity - Min"][cycle] * cycle_days, 0, 0, 0, f"{importer} - Min", ""])
        # Variable flow
        regions_supply_dict[region["Region"]].append(
            [
                1,
                connections_dict[importer_id],
                connections_data_df.loc[importer, "Tariff - fixed"][cycle],
                connections_data_df.loc[importer, "Tariff - variable"][cycle] * unit_conversions_df.loc["mcm"]["MWh"],
                0,
                importer,
                connections_df[connections_df["Name"] == importer]["Origin"].item()
            ]
        )
    
    # Region connections (exports)
    export_list = connections_df[connections_df["Origin Index"] == region_index]["Name"].to_dict()
    for exporter_id, exporter in export_list.items():
        # Min flow
        regions_supply_dict[region["Region"]].append([-1, connections_data_df.loc[exporter, "Capacity - Min"][cycle] * cycle_days, 0, 0, 0, f"{exporter} - Min", ""])
        # Variable flow
        regions_supply_dict[region["Region"]].append([-1, connections_dict[exporter_id], 0, -connections_data_df.loc[exporter, "Tariff - variable"][cycle] * unit_conversions_df.loc["mcm"]["MWh"], 0, exporter, connections_df[connections_df["Name"] == exporter]["Destination"].item()])
    
    '''
    #TODO needs rework
    # Storage
    storage_list = storage_connections_df[storage_connections_df["Region Index"] == region_index]["Name"].to_dict()
    for storage_id, storage in storage_list.items():
        # Min Flow
        print(storage_connections_data_df.loc[storage, "Capacity - Min"][cycle])
        # Variable Flow
        print(storage_connections_dict[storage_id])
    '''

regions_supply_dict

{'Deutschland': [[1, 62, 0, 0, 0, 'Production Deutschland', ''],
  [1, 0.0, 0, 0, 0, 'Import Nor-DE I - Min', ''],
  [1, Import_Nor_DE_I, 0.0, 4884.722222222223, 0, 'Import Nor-DE I', 'Pipe'],
  [1, 0.0, 0, 0, 0, 'Import Nor-DE II - Min', ''],
  [1, Import_Nor_DE_II, 0.0, 6838.611111111111, 0, 'Import Nor-DE II', 'Pipe'],
  [1, 0.0, 0, 0, 0, 'GATE DE - Min', ''],
  [1, GATE_DE, 0.0, 1953.8888888888891, 1, 'GATE DE', 'LNG'],
  [1, 0.0, 0, 0, 0, 'FR-DE - Min', ''],
  [1, FR_DE, 0.0, 976.9444444444446, 0, 'FR-DE', 'Frankreich'],
  [1, 0.0, 0, 0, 0, 'NL-DE - Min', ''],
  [1, NL_DE, 0.0, 2930.8333333333335, 0, 'NL-DE', 'Niederlande'],
  [-1, 0.0, 0, 0, 0, 'DE-FR I - Min', ''],
  [-1, DE_FR_I, 0, -1172.3333333333335, 0, 'DE-FR I', 'Frankreich'],
  [-1, 0.0, 0, 0, 0, 'DE-NL - Min', ''],
  [-1, DE_NL, 0, -1953.8888888888891, 0, 'DE-NL', 'Niederlande'],
  [-1, 0.0, 0, 0, 0, 'DE-CH - Min', ''],
  [-1, DE_CH, 0, -4884.722222222223, 0, 'DE-CH', 'Schweiz'],
  [-1, 0.0, 0, 0, 0, 'DE-FR II - Min', ''

## DONE Model - new

### DONE Initialisation

In [34]:
cost_total = pulp.LpProblem('Cost', pulp.LpMinimize)

### DONE Volume Balancing

#### DONE Supply/Demand Matching

In [35]:
for region_key, region_data in regions_supply_dict.items():
    # Aggregate all supply sources (minus exports) for region
    region_supply = pulp.lpSum([row[0] * row[1] for row in region_data])
    # Add constraints so that supply will exactly match demand
    cost_total += (region_supply <= regions_demand_dict[region_key])
    cost_total += (region_supply >= regions_demand_dict[region_key])

#### DONE Total Piped Volume

In [36]:
for piped_importer_index, piped_importer in piped_importers_df.iterrows():
    # Aggregate all piped import connections
    importer_supply = pulp.lpSum([piped_importers_connections_dict[piped_importers_connection] for piped_importers_connection in piped_importers_connections_df[piped_importers_connections_df["Importer Index"] == piped_importer_index].index.tolist()])
    # Add constraints so that the sum of flows will not exceed production
    cost_total += (importer_supply <= (piped_importers_production_df.loc[piped_importer_index][cycle] - piped_importers_connections_data_df[piped_importers_connections_data_df["Importer"] == piped_importer["Importer"]].xs("Capacity - Min", level = 1, drop_level = False)[cycle].sum()) * cycle_days)

#### LNG Supply

In [37]:
# LNG volume
lng_total = (
    # Demand outside markets
    lng_other_demand_dict[cycle] * unit_conversions_df.loc["Mt LNG"]["mcm"]
    # Demand
    + regions_demand_df[cycle].sum()
    # Production
    - regions_production_df[cycle].sum()
    # Piped Imports - min of available production and max capacity
    - min(
        piped_importers_production_df[cycle].sum(),
        piped_importers_connections_data_df.xs("Capacity - Max", level = 1, drop_level = False)[cycle].sum()
    )
) * unit_conversions_df.loc["mcm"]["Mt LNG"]

# LNG price curve
lng_pricelist = lng_supply_curve_df[cycle].to_frame().reset_index().values

# LNG price at given point
lng_price = lng_pricelist[np.argmin(np.abs(lng_pricelist[0:, 0] - lng_total)), 1] / forex_conversions_df.loc["USD"][cycle] / unit_conversions_df.loc["MMBTU"]["mcm"]

#### Storage - TBD

### Cost Calculations

In [38]:
#TODO add fixed costs
regions_cost_dict = {}
for region_key, region_data in regions_supply_dict.items():
    # Multiply attracted supplies (1, not -1) by their respective tariffs
    # 0 is factor
    # 1 is volume
    # 3 is variable tariff
    # 4 is lng switch
    region_cost = pulp.lpSum([row[0] * row[1] * (max(row[3], 0) + row[4] * lng_price) for row in region_data if row[0] == 1])
    # Append to dictionary
    regions_cost_dict[region_key] = region_cost

# Add up total cost and add as objective    
cost_total += pulp.lpSum([region_data for region_key, region_data in regions_cost_dict.items()])

### DONE Solve

In [39]:
res = cost_total.solve()
assert res == pulp.LpStatusOptimal

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/flo/.pyenv/versions/3.8.6/envs/gasmarketmodel/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/8fbb359d693b43599a7f4879b41274f0-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/8fbb359d693b43599a7f4879b41274f0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 99 RHS
At line 110 BOUNDS
At line 130 ENDATA
Problem MODEL has 10 rows, 19 columns and 64 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10 (0) rows, 18 (-1) columns and 61 (-3) elements
0  Obj 60375.156 Primal inf 232.70001 (4)
9  Obj 31944533
Optimal - objective value 31944533
After Postsolve, objective 31944533, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 31944532.63 - 9 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time 

### Prices

#### Price Sources

In [40]:
model_vars_dict = {}
for model_var in cost_total.variables():
    model_vars_dict[model_var.name] = model_var.value()
model_vars_dict

{'CH_FR': 31.0,
 'DE_CH': 0.0,
 'DE_FR_I': 15.5,
 'DE_FR_II': 15.5,
 'DE_NL': 62.0,
 'Dunkirk_FR': 0.0,
 'FR_CH': 0.0,
 'FR_DE': 0.0,
 'FR_NL': 0.0,
 'GATE_DE': 186.0,
 'GATE_NL': 0.0,
 'Import_Alg_CH': 0.0,
 'Import_Alg_FR': 31.0,
 'Import_Nor_DE_I': 0.0,
 'Import_Nor_DE_II': 0.0,
 'Import_Nor_FR': 0.0,
 'Import_Nor_NL': 15.5,
 'NL_DE': 0.0,
 'NL_FR': 0.0}

In [41]:
regions_sources_dict = {}
regions_prices_dict = {}
for region_key, region_data in regions_supply_dict.items():
    regions_sources_dict[region_key] = []
    # Check if any flows are imports. If so, we can discard exports (see below)
    positive_flag = False
    for region_vars in region_data:
        # Only variable flows
        if isinstance(region_vars[1], pulp.pulp.LpVariable):
            # Only non-zero flows
            if model_vars_dict[region_vars[1].name] != 0 and region_vars[3] != 0 and region_vars[6] != "Pipe":
                # Set flag if this is a positive flow
                if region_vars[3] >= 0:
                    positive_flag = True
                # Source, cost per MWh imported (only variable)
                regions_sources_dict[region_key].append(
                    [
                        region_vars[6],
                        region_vars[3] / unit_conversions_df.loc["mcm"]["MWh"]
                    ]
                )
    # We actually only need to keep exports (negative tariffs) if no positive ones exist (see above)
    if positive_flag == True:
        temp_list = regions_sources_dict[region_key]
        regions_sources_dict[region_key] = [temp_row for temp_row in temp_list if temp_row[1] >= 0]
    regions_prices_dict[region_key] = None
regions_sources_dict

{'Deutschland': [['LNG', 0.2]],
 'Frankreich': [['Deutschland', 0.12000000000000001],
  ['Schweiz', 0.2],
  ['Deutschland', 0.15]],
 'Niederlande': [['Deutschland', 0.2]],
 'Schweiz': [['Frankreich', -0.2]]}

#### Find Prices

In [42]:
# The algorithm always finds the next possible market price (i.e. one with no ambiguity)
# The region is then removed from the dict
# Continuous iteration until all solutions are found

# Start with LNG
regions_prices_dict["LNG"] = lng_price / unit_conversions_df.loc["mcm"]["MWh"]

# Keep looping until all values have been found
while None in regions_prices_dict.values():
    # Take a temporary copy to check that there has been progress in solving this
    temp_dict = regions_sources_dict.copy()
    
    # Iterate over all regions
    for region_key, region_prices in regions_sources_dict.items():
        for price_index, region_price in enumerate(region_prices):
            # Check if market price has already been found
            if isinstance(region_price, list):
                if regions_prices_dict[region_price[0]] != None:
                    regions_sources_dict[region_key][price_index] = regions_prices_dict[region_price[0]] + region_price[1]
        
        # Now check if all values have been found, and if so save it. Then delete entry from dict and restart
        if all(isinstance(region_price, (int, float)) for region_price in region_prices):
            regions_prices_dict[region_key] = max(region_prices)
            del regions_sources_dict[region_key]
            break
    
    # If no progress has been made
    if regions_sources_dict == temp_dict:
        # Iterate over remaining regions
        for region_key, region_prices in regions_sources_dict.items():
            # Find first instance of an unresolved price
            for region_price_index, region_price in enumerate(region_prices):
                if isinstance(region_price, list):
                    # Delete it
                    del regions_sources_dict[region_key][region_price_index]
                    break
                else:
                    continue
            break
        
regions_prices_dict

{'Deutschland': 17.398958440798808,
 'Frankreich': 17.548958440798806,
 'Niederlande': 17.598958440798807,
 'Schweiz': 17.348958440798807,
 'LNG': 17.19895844079881}

## DONE Dataframe Output

In [58]:
pd.concat(
    [
        output_piped_importers_df,
        output_lng_importers_df
    ]
)

,Flow Type,Type,Importer,Region,Aug-21,Terminal
Import Nor-DE I,Piped Imports,Piped Imports,Norwegen,Deutschland,0.0,NaN
Import Nor-FR,Piped Imports,Piped Imports,Norwegen,Frankreich,0.0,NaN
Import Nor-NL,Piped Imports,Piped Imports,Norwegen,Niederlande,1.0,NaN
Import Alg-FR,Piped Imports,Piped Imports,Algerien,Frankreich,1.0,NaN
Import Alg-CH,Piped Imports,Piped Imports,Algerien,Schweiz,0.0,NaN
Import Nor-DE II,Piped Imports,Piped Imports,Norwegen,Deutschland,0.0,NaN
GATE NL,LNG Imports,LNG Imports,NaN,Niederlande,0.0,GATE
GATE DE,LNG Imports,LNG Imports,NaN,Deutschland,6.0,GATE
Dunkirk FR,LNG Imports,LNG Imports,NaN,Frankreich,0.0,Dunkirk


### Piped Imports

In [57]:
for piped_importer_id, piped_importer in piped_importers_connections_dict.items():
    piped_importer_name = piped_importers_connections_df.loc[piped_importer_id]["Name"]
    # Only in first cycle
    if cycle_index == 0:
        output_piped_importers_df.at[
            piped_importer_name,
            "Importer"
        ] = piped_importers_connections_df.loc[piped_importer_id]["Importer"]
        output_piped_importers_df.at[
            piped_importer_name,
            "Region"
        ] = piped_importers_connections_df.loc[piped_importer_id]["Region"]
    output_piped_importers_df.at[
        piped_importer_name,
        cycle
    ] = piped_importer.value() / cycle_days + piped_importers_connections_data_df.loc[piped_importer_name, "Capacity - Min"][cycle]
output_piped_importers_df

,Flow Type,Type,Importer,Region,Aug-21
Import Nor-DE I,Piped Imports,Piped Imports,Norwegen,Deutschland,0.0
Import Nor-FR,Piped Imports,Piped Imports,Norwegen,Frankreich,0.0
Import Nor-NL,Piped Imports,Piped Imports,Norwegen,Niederlande,1.0
Import Alg-FR,Piped Imports,Piped Imports,Algerien,Frankreich,1.0
Import Alg-CH,Piped Imports,Piped Imports,Algerien,Schweiz,0.0
Import Nor-DE II,Piped Imports,Piped Imports,Norwegen,Deutschland,0.0


### LNG Imports

In [44]:
for lng_importer_id, lng_importer in lng_importers_connections_dict.items():
    lng_importer_name = lng_importers_connections_df.loc[lng_importer_id]["Name"]
    # Only in first cycle
    if cycle_index == 0:
        output_lng_importers_df.at[
            lng_importer_name,
            "Terminal"
        ] = lng_importers_connections_df.loc[lng_importer_id]["Terminal"]
        output_lng_importers_df.at[
            lng_importer_name,
            "Region"
        ] = lng_importers_connections_df.loc[lng_importer_id]["Region"]
    output_lng_importers_df.at[
        lng_importer_name,
        cycle
    ] = lng_importer.value() / cycle_days + lng_importers_connections_data_df.loc[lng_importer_name, "Capacity - Min"][cycle]
output_lng_importers_df

,Terminal,Region,Aug-21
GATE NL,GATE,Niederlande,0.0
GATE DE,GATE,Deutschland,6.0
Dunkirk FR,Dunkirk,Frankreich,0.0


### Region Connections

In [45]:
for connection_id, connection in connections_dict.items():
    connection_name = connections_df.loc[connection_id]["Name"]
    # Only in first cycle
    if cycle_index == 0:
        output_connections_df.at[
            connection_name,
            "Origin"
        ] = connections_df.loc[connection_id]["Origin"]
        output_connections_df.at[
            connection_name,
            "Destination"
        ] = connections_df.loc[connection_id]["Destination"]
    output_connections_df.at[
        connection_name,
        cycle
    ] = connection.value() / cycle_days + connections_data_df.loc[connection_name, "Capacity - Min"][cycle]
output_connections_df

,Origin,Destination,Aug-21
DE-FR I,Deutschland,Frankreich,0.5
FR-DE,Frankreich,Deutschland,0.0
DE-NL,Deutschland,Niederlande,2.0
NL-DE,Niederlande,Deutschland,0.0
DE-CH,Deutschland,Schweiz,0.0
FR-NL,Frankreich,Niederlande,0.0
NL-FR,Niederlande,Frankreich,0.0
FR-CH,Frankreich,Schweiz,0.0
CH-FR,Schweiz,Frankreich,1.0
DE-FR II,Deutschland,Frankreich,0.5


### Storage

### Prices

In [46]:
# Only in first cycle
if cycle_index == 0:
    output_prices_df = output_prices_df.from_dict(regions_prices_dict, orient = "index", columns = [cycle])
else:
    output_prices_df[cycle] = pd.Series(regions_prices_dict)
output_prices_df

,Aug-21
Deutschland,17.398958
Frankreich,17.548958
Niederlande,17.598958
Schweiz,17.348958
LNG,17.198958


### Regional Supply Mix

In [47]:
output_supply_mix_dict = {}
output_supply_mix_df[cycle] = 0

for region_key, region_data in regions_demand_dict.items():
    demand = output_demand_df.loc[region_key][cycle]
    
    output_supply_mix_dict["Production"] = output_production_df.loc[region_key][cycle]
    output_supply_mix_dict["Piped Imports"] = output_piped_importers_df[output_piped_importers_df["Region"] == region_key][cycle].sum()
    output_supply_mix_dict["LNG Imports"] = output_lng_importers_df[output_lng_importers_df["Region"] == region_key][cycle].sum()
    output_supply_mix_dict["Imports"] = output_connections_df[output_connections_df["Destination"] == region_key][cycle].sum()
    
    supply = sum(output_supply_mix_dict.values())
    
    for mix_key, mix_value in output_supply_mix_dict.items():
        output_supply_mix_df.loc[(region_key, mix_key), cycle] = mix_value/supply*demand
        
output_supply_mix_df

Aug-21
Region      Type                 
Deutschland Production       1.25
            Piped Imports    0.00
            LNG Imports      3.75
            Imports          0.00
Frankreich  Production       2.00
            Piped Imports    1.00
            LNG Imports      0.00
            Imports          2.00
Niederlande Production       2.00
            Piped Imports    1.00
            LNG Imports      0.00
            Imports          2.00
Schweiz     Production       8.00
            Piped Imports    0.00
            LNG Imports      0.00
            Imports          0.00

## File Output

In [48]:
# Dictionary with all output dataframes.
# Format Label: [dataframe, total_row_switch, row offset]
output_dict = {
    "Demand": [output_demand_df.sort_index(), True, 0],
    "Production": [output_production_df.sort_index(), True, 0],
    "Price": [output_prices_df.sort_index(), False, 0],
    "LNG": [output_lng_importers_df.sort_values(["Terminal", "Region"]), True, 8],
    "Piped Imports": [output_piped_importers_df.sort_values(["Importer", "Region"]), True, 12],
    "Connections": [output_connections_df.sort_values(["Origin", "Destination"]), False, 12],
    "Supply Mix": [output_supply_mix_df.sort_index().reset_index().set_index("Region"), False, 9]
}

In [49]:
# Copy template
shutil.copyfile(output_template_path, output_path)

# Excel Writer
wb = openpyxl.load_workbook(output_path)

for output_key, output_data in output_dict.items():
    ws = wb[output_key]
    # Add total row if required
    if output_data[1]:
        index_col = output_data[0].append(output_data[0].sum(numeric_only = True).rename("Total")).index.tolist()
        value_cols = output_data[0].append(output_data[0].sum(numeric_only = True).rename("Total")).values.tolist()
    else:
        index_col = output_data[0].index.tolist()
        value_cols = output_data[0].values.tolist()
    index_row = output_data[0].columns.tolist()
    for row_index, row in enumerate(value_cols):
        ws.cell(row = row_index + 6 + output_data[2], column = 1, value = index_col[row_index])
        for col_index, col in enumerate(row):
            ws.cell(row = row_index + 6 + output_data[2], column = col_index + 2, value = col)
            ws.cell(row = 5 + output_data[2], column = col_index + 2, value = index_row[col_index])

# Save
wb.save(output_path)

## Output

### Flows

In [50]:
# Map with arrows for flows

### Prices

In [51]:
# Map with market prices

### Supply/Demand Balance

In [52]:
# Pie chart per region showing (adjusted) production, piped imports, LNG, (net imports)